## Challenge

--- Day 14: Disk Defragmentation ---

Suddenly, a scheduled job activates the system's disk defragmenter. Were the situation different, you might sit and watch it for a while, but today, you just don't have that kind of time. It's soaking up valuable system resources that are needed elsewhere, and so the only option is to help it finish its task as soon as possible.

The disk in question consists of a 128x128 grid; each square of the grid is either free or used. On this disk, the state of the grid is tracked by the bits in a sequence of knot hashes.

A total of 128 knot hashes are calculated, each corresponding to a single row in the grid; each hash contains 128 bits which correspond to individual grid squares. Each bit of a hash indicates whether that square is free (0) or used (1).

The hash inputs are a key string (your puzzle input), a dash, and a number from 0 to 127 corresponding to the row. For example, if your key string were flqrgnkx, then the first row would be given by the bits of the knot hash of flqrgnkx-0, the second row from the bits of the knot hash of flqrgnkx-1, and so on until the last row, flqrgnkx-127.

The output of a knot hash is traditionally represented by 32 hexadecimal digits; each of these digits correspond to 4 bits, for a total of 4 * 32 = 128 bits. To convert to bits, turn each hexadecimal digit to its equivalent binary value, high-bit first: 0 becomes 0000, 1 becomes 0001, e becomes 1110, f becomes 1111, and so on; a hash that begins with a0c2017... in hexadecimal would begin with 10100000110000100000000101110000... in binary.

Continuing this process, the first 8 rows and columns for key flqrgnkx appear as follows, using # to denote used squares, and . to denote free ones:

##.#.#..-->
.#.#.#.#   
....#.#.   
#.#.##.#   
.##.#...   
##..#..#   
.#...#..   
##.#.##.-->
|      |   
V      V   

In this example, 8108 squares are used across the entire 128x128 grid.

Given your actual key string, how many squares are used?

Your puzzle input is oundnydw.

## Part 1

In [91]:
import operator
from functools import reduce

length_suffix = [17, 31, 73, 47, 23]

# strings = [f'flqrgnkx-{i}' for i in range(128)]
strings = [f'oundnydw-{i}' for i in range(128)]
row_binaries = []

for string in strings:
    lengths = [int(l) for l in map(ord, string)]
    lengths += length_suffix

    string_list = [i for i in range(256)]
    current_position = 0
    skip_size = 0

    for rnd in range(64):
        for length in lengths:

            sub_list = {}

            for sub_ind in range(length):

                pos = int((current_position + sub_ind) % len(string_list))
                val = string_list[pos]

                sub_list[pos] = val

            rev_list = {ind: val for ind, val in zip(
                list(sub_list.keys()), list(sub_list.values())[::-1])}

            for i, val in rev_list.items():
                string_list[i] = val

            current_position = int(
                (current_position + int(length + skip_size)) % len(string_list)
            )
            skip_size += 1

    chunk_list = [string_list[(step*16):((step*16)+16)] for step in range(16)]
    dense_hash = [reduce(operator.xor, chunk, 0) for chunk in chunk_list]
    hexes = ['{:02x}'.format(d) for d in dense_hash]
    hex_string = ''.join(hexes)
#     binaries = []
    
#     for h in hexes:
        
#         byte = []
        
#         for c in h:
#             b = bin(int(c,16))[2:].zfill(4)
#             binaries.append(b)
        
#         sorted(byte)
        
#         for bit in byte:
#             binaries.append(bit)
        
    binary = [bin(int(hex_string, 16))[2:].zfill(8) for hex_string in hexes]
    row_binaries.append(''.join(binary))

used_spaces = 0

for binary in row_binaries:
    used_spaces += sum([int(b) for b in binary])

print(used_spaces)

8106


In [85]:
h = 'a0c2017'

for hex_string in h:
    print(bin(int(hex_string, 16))[2:].zfill(4))
# binary = sorted([bin(int(hex_string, 16))[2:].zfill(4) for hex_string in h], reverse=True)
# print(''.join(binary))

1010
0000
1100
0010
0000
0001
0111


In [ ]:
10100000110000100000000101110000

In [74]:
row_lengths = []

for i, row in enumerate(row_binaries):
    if len(row) < 128:
        row_lengths.append(i)

print(row_lengths)

[]


In [75]:
row_binaries[2]

'00001010000100110011000101000000010100110110101001110110011101111000000110110010101101111101111111101000111010101111001111111010'

## Part 2

In [92]:
def search_adjacent(cell):
    
    row, col = cell
    
    up = (row-1, col)
    down = (row+1, col)
    left = (row, col-1)
    right = (row, col+1)
    
    region = []
    
    for adj in [down, right, up, left]:
        
        if adj in cells and not adj in scanned_cells:
            
            scanned_cells.append(adj)
            
            if row_binaries[adj[0]][adj[1]] == '1':
                
                region.append(adj)
                
                region += search_adjacent(adj)
    
    return region

cells = [(r, c) for r in range(128) for c in range(128)]

scanned_cells = []
regions = {}
region_num = 0

for cell in cells:
    
    row, col = cell
    
    if not cell in scanned_cells:
        
        scanned_cells.append(cell)
    
        if row_binaries[row][col] == '1':

            regions[region_num] = [cell]

#             cells.remove(cell)

            regions[region_num] += search_adjacent(cell)

            region_num += 1
        

print(region_num)
print(len(regions))

1164
1164


In [52]:
regions

{0: [(0, 0), (0, 1), (1, 1)],
 1: [(0, 3), (1, 3)],
 2: [(0, 5), (1, 5)],
 3: [(0, 8),
  (1, 8),
  (2, 8),
  (2, 9),
  (1, 9),
  (0, 9),
  (0, 10),
  (1, 10),
  (0, 11),
  (1, 7)],
 4: [(0, 13),
  (0, 14),
  (1, 14),
  (2, 14),
  (2, 13),
  (3, 13),
  (4, 13),
  (5, 13),
  (4, 12),
  (3, 12),
  (2, 12),
  (1, 12),
  (2, 11),
  (3, 11),
  (3, 10),
  (4, 10),
  (5, 10),
  (5, 9),
  (6, 9),
  (7, 9),
  (8, 9),
  (8, 8),
  (9, 8),
  (10, 8),
  (11, 8),
  (11, 7),
  (11, 6),
  (12, 6),
  (12, 5),
  (11, 5),
  (11, 4),
  (11, 3),
  (10, 3),
  (9, 3),
  (8, 3),
  (7, 3),
  (8, 2),
  (8, 1),
  (7, 1),
  (6, 1),
  (5, 1),
  (4, 1),
  (4, 2),
  (3, 2),
  (5, 0),
  (7, 0),
  (8, 0),
  (9, 0),
  (9, 1),
  (12, 3),
  (11, 2),
  (11, 1),
  (13, 5),
  (13, 4),
  (14, 4),
  (15, 4),
  (15, 5),
  (16, 5),
  (17, 5),
  (18, 5),
  (19, 5),
  (19, 4),
  (18, 4),
  (17, 4),
  (17, 3),
  (16, 3),
  (16, 2),
  (17, 2),
  (18, 2),
  (19, 2),
  (18, 1),
  (17, 1),
  (17, 0),
  (18, 3),
  (20, 4),
  (18, 6),
  

In [70]:
grid = [['.' for _ in range(128)] for _ in range(128)]

for region, cells in regions.items():
    
    for cell in cells:
        r, c = cell
        grid[r][c] = str(region + 1)

with open('grid.txt', 'w') as f:
    print('\n'.join([''.join(row) for row in grid]), file=f)